## Machine Translation (English to French)



- **Preprocess** - converted text to sequence of integers.
- **Models** - constructed 5 models: simple RNN, embedded RNN, BRNN, Encoder-Decoder, and all combined
- **Prediction** Run the model on English text

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4934978347321389734
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357433344
locality {
  bus_id: 1
}
incarnation: 9046379477279707207
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
A small subset of data from [WMT](http://www.statmt.org/).

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


### Vocabulary

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess
### Tokenize

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer(num_words=None, char_level=False)
    tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)
    return sequences, tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if (length is None):
        padded_seq = pad_sequences(x, maxlen = max([len(sentence) for sentence in x]), padding="post", truncating="post")
    else:
        padded_seq = pad_sequences(x, maxlen = length, padding="post", truncating="post")
                                   
    return padded_seq

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models

- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

### Model 1 - basic RNN model
![RNN](images/rnn.png)


In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    lr = 1e-2
    
    inputs = Input(input_shape[1:])
    hidden_layer = GRU(output_sequence_length, return_sequences=True)(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(hidden_layer)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(lr), metrics=["acc"])
        
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

simple_rnn_model.summary()

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 21)            1449      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           7568      
Total params: 9,017
Trainable params: 9,017
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 7s 64us/step - loss: 2.9175 - acc: 0.4412 - val_loss: nan - val_acc: 0.4822
Epoch 2/10
110288/110288 [==============================] - 6s 57us/step - loss: 2.2430 - acc: 0.4937 - val_loss: nan - val_acc: 0.5142
Epoch 3/10
110288/110288 [==============================] - 6s 57us/step - loss: 

### Model 2: Embedding
![RNN](images/embedding.png)

In [21]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    lr = 0.01
    
    inputs = Input(input_shape[1:])
    embedding_layer = Embedding(input_dim=english_vocab_size, output_dim=output_sequence_length)(inputs)
    hidden_layer = GRU(output_sequence_length, return_sequences=True)(embedding_layer)
    outputs = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(hidden_layer)
    
    model = Model(inputs, outputs)
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(lr), 
                 metrics=['acc'])
    
    return model

tests.test_embed_model(embed_model)


# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
embed_rnn_model = embed_model(tmp_x.shape, preproc_french_sentences.shape[1], len(english_tokenizer.word_index)+1, 
                             len(french_tokenizer.word_index)+1)
embed_rnn_model.summary()
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 21)                0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 21, 21)            4200      
_________________________________________________________________
gru_17 (GRU)                 (None, 21, 21)            2709      
_________________________________________________________________
time_distributed_16 (TimeDis (None, 21, 345)           7590      
Total params: 14,499
Trainable params: 14,499
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 73us/step - loss: 2.8838 - acc: 0.4601 - val_loss: 1.9002 - val_acc: 0.5448
Epoch 2/10
110288/110288 [==============================] - 7s 65us/step - loss:

### Model 3: Bidirectional RNNs
![RNN](images/bidirectional.png)

In [22]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    lr = 0.01
    
    inputs = Input(input_shape[1:])
    hidden_layer = Bidirectional(GRU(output_sequence_length, return_sequences=True))(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(hidden_layer)
    
    model = Model(inputs, outputs)
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(lr), metrics=["acc"])
    
    return model

tests.test_bd_model(bd_model)

# Train and Print prediction(s)

# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)
bd_rnn_model.summary()
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 21, 42)            2898      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 21, 345)           14835     
Total params: 17,733
Trainable params: 17,733
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 97us/step - loss: 2.5228 - acc: 0.4970 - val_loss: 1.7429 - val_acc: 0.5753
Epoch 2/10
110288/110288 [==============================] - 10s 86us/step - loss: 1.5800 - acc: 0.5932 - val_loss: 1.4814 - val_acc: 0.6081
Epoch 3/10
110288/110288 [==============================] - 10s 87us/st

### Model 4: Encoder-Decoder

In [23]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    lr = 0.01
    latent_dim = 128
    
    # encoder
    enc_inputs = Input(input_shape[1:])
    enc_gru = GRU(output_sequence_length)(enc_inputs)
    enc_outputs = Dense(latent_dim, activation="relu")(enc_gru)
    
    # decoder
    dec_inputs = RepeatVector(output_sequence_length)(enc_outputs) # repeat enc output by many times
    dec_gru = GRU(latent_dim, return_sequences=True)(dec_inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(dec_gru)
    
    model = Model(enc_inputs, outputs)
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(lr), metrics=["acc"])
    
    return model

tests.test_encdec_model(encdec_model)

# Train and Print prediction(s)

# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdec_rnn_model = encdec_model(tmp_x.shape, preproc_french_sentences.shape[1],
                                len(english_tokenizer.word_index) + 1, len(french_tokenizer.word_index) + 1)
encdec_rnn_model.summary()
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
gru_22 (GRU)                 (None, 21)                1449      
_________________________________________________________________
dense_22 (Dense)             (None, 128)               2816      
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 21, 128)           0         
_________________________________________________________________
gru_23 (GRU)                 (None, 21, 128)           98688     
_________________________________________________________________
time_distributed_20 (TimeDis (None, 21, 345)           44505     
Total params: 147,458
Trainable params: 147,458
Non-trainable params: 0
_________________________________________________________________
Trai

### Model 5: Combination of Previous Models

In [32]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    lr = 0.01
    latent_dim = 128
    
    inputs = Input(input_shape[1:])
    embedding_layer = Embedding(input_dim=english_vocab_size,
                               output_dim=output_sequence_length,
                               mask_zero=False)(inputs)
    bd_layer = Bidirectional(GRU(output_sequence_length))(embedding_layer)
    enc_layer = Dense(latent_dim, activation="relu")(bd_layer)
    dec_layer = RepeatVector(output_sequence_length)(enc_layer)
    output_layer = Bidirectional(GRU(latent_dim, return_sequences=True))(dec_layer)
    outputs = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(output_layer)
    
    model = Model(inputs, outputs)
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(lr),
                 metrics=["acc"])
    
    return model

tests.test_model_final(model_final)

## Prediction

In [33]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using model_final
    model = model_final(x.shape, 
                        y.shape[1], 
                        len(x_tk.word_index)+1, 
                        len(y_tk.word_index)+1)
    
    model.summary()
    model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)

    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 15)                0         
_________________________________________________________________
embedding_16 (Embedding)     (None, 15, 21)            4200      
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 42)                5418      
_________________________________________________________________
dense_38 (Dense)             (None, 128)               5504      
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 21, 128)           0         
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 21, 256)           197376    
_________________________________________________________________
time_distributed_28 (TimeDis (None, 21, 345)           88665     
Total para

### Generate the html

In [34]:
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 363697 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328614 bytes to machine_translation-zh.html']